In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import division, unicode_literals

In [3]:
import os 
from pprint import pprint

The aim of the streamm package is to design a project with numerous calculations locally then run them on local and remote resources and collect the output for analysis, thus facilitating high-throughput computational material design.

To accoplish this directory structures are contained in a resource. Resources, structures and forcefields are contained within a calculation. Sets of calculations are contained within a project

So let's first create a resource object that we will use to set the directory locations of all the subsequent calculation objects 

In [4]:
import streamm.calculations.resource as resource  

In [5]:
res_i = resource.Resource('local')

This sets the current working directory as the root/home directory by default

In [6]:
pprint(res_i.dir)

{u'home': '/Users/tkemper/Development/streamm-tools/examples',
 u'launch': u'/Users/tkemper/Development/streamm-tools/examples/scratch',
 u'materials': u'/Users/tkemper/Development/streamm-tools/examples/materials',
 u'scratch': u'/Users/tkemper/Development/streamm-tools/examples/scratch',
 u'scripts': u'/Users/tkemper/Development/streamm-tools/examples/scripts',
 u'storage': u'/Users/tkemper/Development/streamm-tools/examples/storage',
 u'templates': u'/Users/tkemper/Development/streamm-tools/examples/templates'}


Let's create a new home directory called 'example_proj'

In [7]:
EXAMPLE_DIR = res_i.dir['home']

In [8]:
new_root = os.path.join(EXAMPLE_DIR,'example_proj')

In [9]:
res_i.set_home(new_root)

In [22]:
pprint(res_i.dir)

{u'home': u'/Users/tkemper/Development/streamm-tools/examples/example_proj',
 u'launch': u'/Users/tkemper/Development/streamm-tools/examples/example_proj/scratch',
 u'materials': '/Users/tkemper/Development/streamm-tools/examples',
 u'scratch': u'/Users/tkemper/Development/streamm-tools/examples/example_proj/scratch',
 u'scripts': u'/Users/tkemper/Development/streamm-tools/examples/example_proj/scripts',
 u'storage': u'/Users/tkemper/Development/streamm-tools/examples/example_proj/storage',
 u'templates': u'/Users/tkemper/Development/streamm-tools/examples/example_proj/templates'}


However, we want to use structures from our previous structures and forcefields examples, so let's set the materials directory to examples/

In [11]:
res_i.dir['materials'] = EXAMPLE_DIR

This also contains the properties dictionary, which can be used to write .pbs scripts on clusters

In [12]:
pprint(res_i.properties)

{u'allocation': u'',
 u'exe_command': u'./',
 u'feature': u'24core',
 u'nodes': 1,
 u'nproc': 1,
 u'pmem': 1500,
 u'ppn': 1,
 u'queue': u'batch',
 u'walltime': 24}


By default the resource type is 'local'; however, setting type to 'ssh' will invoke an scp command when copying files

Okay create the directories we need for our calculation

In [13]:
res_i.make_dir()

Now we should have a directory 'example_proj/' with materials, scratch,   scripts,   storage and   templates directories 

We can create a gaussian calculation

In [14]:
import streamm.calculations.gaussian as gaussian  

In [15]:
calc_i = gaussian.Gaussian('methane_HF')

Set the resource and all the directories 

In [23]:
calc_i.set_resource(res_i)

In [28]:
pprint(calc_i.dir)

{u'home': u'/Users/tkemper/Development/streamm-tools/examples/example_proj',
 u'launch': u'/Users/tkemper/Development/streamm-tools/examples/example_proj/scratch/methane_HF/',
 u'materials': '/Users/tkemper/Development/streamm-tools/examples',
 u'scratch': u'/Users/tkemper/Development/streamm-tools/examples/example_proj/scratch/methane_HF/',
 u'scripts': u'/Users/tkemper/Development/streamm-tools/examples/example_proj/scripts',
 u'storage': u'/Users/tkemper/Development/streamm-tools/examples/example_proj/storage/methane_HF/',
 u'templates': u'/Users/tkemper/Development/streamm-tools/examples/example_proj/templates'}


Make the calculation directories

In [25]:
calc_i.make_dir()

Let's assign a structure to this calculation 

First copy the .xyz file from the materials directory to our scratch directory using the cp_file() function.

This takes an type and key to set the calc_i.files[type][key] dictionary 

In [26]:
file_type = 'input'
file_key = 'xyz'
file_name = "methane.xyz"
from_dirkey = 'materials'
to_dirkey = 'scratch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

Generally the materials directory is thought to contain a repository of material files, and local versions in the scratch directory should be made in case modifications are necessary 

Change to the scratch directory 

In [29]:
pprint(calc_i.dir['scratch'])

u'/Users/tkemper/Development/streamm-tools/examples/example_proj/scratch/methane_HF/'


In [30]:
os.chdir(calc_i.dir['scratch'])

Read in methane .xyz file from the structures example

In [32]:
calc_i.strucC.read_xyz('methane.xyz')

In [34]:
print calc_i.strucC.n_particles

5


Now that we have a structure and parameters for each interaction we can create an input file for a simulation

To write out input files we will use the templates provided in the streamm package

Set the template dir dictionary entry to the location of templates directory 

In [43]:
calc_i.dir['templates'] =  os.path.join(EXAMPLE_DIR,'..','templates','')

In [44]:
print calc_i.dir['templates']

/Users/tkemper/Development/streamm-tools/examples/../templates/


Get the bash run script for gaussian. By setting the file_key to run, this will be the script that exicuted when the run() function is called

In [45]:

file_type = 'templates'
file_key = 'run'
file_name = "gaussian.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)


Get the .com template

In [47]:

file_type = 'templates'
file_key = 'com'
file_name = "gaussian.com"
from_dirkey = 'templates'
to_dirkey = 'scratch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)


Make sure we are in the scratch directory

In [49]:
pprint(os.getcwd())

'/Users/tkemper/Development/streamm-tools/examples/example_proj/scratch/methane_HF'


Load the template files into memory 

In [51]:
calc_i.load_str('templates','com')        
calc_i.load_str('templates','run')

Set the properties strings in the template files to have the values we want

In [52]:
calc_i.properties['commands'] = 'HF/3-21G SP'
calc_i.properties['charge'] = 0
calc_i.properties['spin_mult'] = 1
calc_i.properties['coord'] = calc_i.strucC.write_coord()

Replace the strings in the files['input']['com']

In [54]:
calc_i.replacewrite_prop('com','input','com','%s.com'%(calc_i.tag))

Add the name of the .com file to the properties, and replace the strings in the files['input']['run']

In [55]:
calc_i.properties['input_com'] = calc_i.files['input']['com']
calc_i.replacewrite_prop('run','scripts','run','%s.sh'%(calc_i.tag))

Save a .json file in the home directory 

In [57]:
os.chdir(calc_i.dir['home'])
calc_i.dump_json()

Go to scratch directory and see if there is a completed output file for the calculation

In [59]:
os.chdir(calc_i.dir['scratch'])
calc_i.check()

Check the status 

In [64]:
pprint("Calculation:{} has status:{}".format(calc_i.tag,calc_i.meta['status']))

u'Calculation:methane_HF has status:written'


Set the forcefield particletypes 

In [ ]:

for pkey,p in mol_i.particles.iteritems():
    if( p.symbol == 'C' ):
        p.ffkey = 'CT'
    elif( p.symbol == 'H' ):
        p.ffkey = 'HC'

In [ ]:
mol_i.bonded_nblist = mol_i.guess_nblist(0,radii_buffer=1.25)

Find bonds and bond angles based on neighbor list 

In [ ]:
mol_i.bonded_bonds()
mol_i.bonded_angles()

Copy the pickled forcefield parameter file to scratch and read it in

In [35]:
file_type = 'input'
file_key = 'param'
file_name = "oplsaa.pkl"
from_dirkey = 'materials'
to_dirkey = 'scratch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [36]:
import streamm.forcefields.container as container 

In [37]:
calc_i.paramC = container.read_pickle('oplsaa')

Use the set_ffparam() function to iterate through the structure container and set parameters based on ffkeys

In [38]:
calc_i.set_ffparam()

TypeError:  No particle parameters were found for atom type None 

So let's first create a project

In [ ]:
import streamm.calculations.project as project  

This object has dictionaries to help keep track of the components of a calculation such as structures, forcefields and files

Read in the parameters we created in the forcefield example

In [ ]:
print paramC

Now we can set the particles, bonds and bond angles of the molecule to have parameters associated with the ffkey

Now we can combine the structure and the forcefield parameters using the Calculation object

In [ ]:
calc_i = calculation.Calculation('methane_oplsaa')

Set the structure container and forcefield container

In [ ]:
calc_i.strucC = mol_i
calc_i.paramC = paramC